In [1]:
import pandas as pd
import numpy as np

In [3]:
np.random.seed(0)

test_clustering_file = 'clustering_table.tsv'
test_blacklist_samples_file = 'blacklist_samples.txt'

test_clustering_df = pd.DataFrame(columns=['SAMPLE_NAME', 'CLUSTER_1', 'CLUSTER_2', 'CLUSTER_3'])
test_num_clusters = 3
test_num_samples = 10000
test_sample_names = np.array(['sample_{:d}'.format(i) for i in range(1, test_num_samples + 1)])
test_unnorm_responsibilities = np.random.random(size=(test_num_samples, test_num_clusters))
test_responsibilities = test_unnorm_responsibilities / np.sum(test_unnorm_responsibilities, axis=1)[:, np.newaxis]
test_clustering_df['SAMPLE_NAME'] = test_sample_names
test_clustering_df[['CLUSTER_1', 'CLUSTER_2', 'CLUSTER_3']] = test_responsibilities
test_clustering_df.to_csv(test_clustering_file, sep='\t', index=False)

test_blacklist_fraction = 0.05
test_blacklist_samples = test_sample_names[1 == np.random.choice([0, 1], 
                                                                 p=[1 - test_blacklist_fraction, test_blacklist_fraction], 
                                                                 size=test_num_samples)]
np.savetxt(test_blacklist_samples_file, test_blacklist_samples, fmt='%s')